In [2]:
import os
import csv
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel
from skimage.transform import resize
from scipy.fftpack import dct as scipy_dct
import cv2
from skimage.feature import hog

TARGET_SIZE = (128, 128)

kernels = [
    np.real(gabor_kernel(frequency, theta=theta / 4.0 * np.pi, sigma_x=sigma, sigma_y=sigma))
    for theta in range(4)
    for sigma in (1, 3)
    for frequency in (0.05, 0.25)
]

def load_and_preprocess_image(image_path):
    image = img_as_float(io.imread(image_path, as_gray=True)) 
    resized_image = resize(image, TARGET_SIZE, anti_aliasing=True)
    return resized_image

def compute_gabor(image, kernels):
    feats = np.array([(ndi.convolve(image, kernel, mode='wrap').mean(), ndi.convolve(image, kernel, mode='wrap').var())
                      for kernel in kernels])
    return feats.flatten()

def compute_dct(image, num_coeffs=100):
    dct_flat = scipy_dct(scipy_dct(image, axis=0, norm='ortho'), axis=1, norm='ortho').flatten()
    return dct_flat[:num_coeffs]

def compute_ft(image, num_coeffs=100):
    fft_magnitude = np.abs(np.fft.fft2(image)).flatten()
    return np.sort(fft_magnitude)[:num_coeffs]

def compute_phog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), pyramid_levels=3, num_coeffs=100):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    pyramid_features = []
    for level in range(pyramid_levels):
        scaled_image = cv2.resize(image, (0, 0), fx=1 / (2 ** level), fy=1 / (2 ** level))
        hog_features = hog(scaled_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=False, feature_vector=True)
        pyramid_features.extend(hog_features)
    return np.array(pyramid_features)[:num_coeffs]


csv_file_path = 'image_features.csv'

# Dossier de base contenant les sous-dossiers
base_dir = os.path.expanduser("~/Desktop/LR_grabor/train")
sub_dirs = ["NORMAL", "PNEUMONIA"]

# Ouvrir le fichier CSV en mode écriture
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Numéro d'image", "Gabor", "DCT", "Fourier", "PHOG", "Étiquette"])

    for sub_dir in sub_dirs:
        folder_path = os.path.join(base_dir, sub_dir)
        label = 0 if sub_dir == "NORMAL" else 1

        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(folder_path, filename)
                    
                    # Charger et prétraiter l'image
                    image = load_and_preprocess_image(image_path)
                    
                    # Calculer les caractéristiques
                    gabor_feats = compute_gabor(image, kernels)
                    dct_feats = compute_dct(image, num_coeffs=20)
                    ft_feats = compute_ft(image, num_coeffs=20)
                    phog_feats = compute_phog(image, num_coeffs=20)
                    
                    gabor_mean = np.mean(gabor_feats)
                    dct_mean = np.mean(dct_feats)
                    ft_mean = np.mean(ft_feats)
                    phog_mean = np.mean(phog_feats)

                    
                    # Numéro de l'image
                    numero_image = filename.split('_')[-1].split('.')[0]
                    
                    # Écrire la ligne dans le fichier CSV
                    writer.writerow([numero_image, gabor_mean, dct_mean, ft_mean, phog_mean, label])
                    print(f"Processed {filename}")

print(f"Image features saved to '{csv_file_path}'.")

Processed NORMAL_1109.jpg
Processed NORMAL_458.jpg
Processed NORMAL_1135.jpg
Processed NORMAL_464.jpg
Processed NORMAL_302.jpg
Processed NORMAL_316.jpg
Processed NORMAL_1121.jpg
Processed NORMAL_470.jpg
Processed NORMAL_55.jpg
Processed NORMAL_882.jpg
Processed NORMAL_896.jpg
Processed NORMAL_41.jpg
Processed NORMAL_128.jpg
Processed NORMAL_69.jpg
Processed NORMAL_100.jpg
Processed NORMAL_666.jpg
Processed NORMAL_1337.jpg
Processed NORMAL_672.jpg
Processed NORMAL_1323.jpg
Processed NORMAL_114.jpg
Processed NORMAL_96.jpg
Processed NORMAL_841.jpg
Processed NORMAL_699.jpg
Processed NORMAL_855.jpg
Processed NORMAL_82.jpg
Processed NORMAL_869.jpg
Processed NORMAL_1243.jpg
Processed NORMAL_712.jpg
Processed NORMAL_1257.jpg
Processed NORMAL_706.jpg
Processed NORMAL_935.jpg
Processed NORMAL_921.jpg
Processed NORMAL_1280.jpg
Processed NORMAL_909.jpg
Processed NORMAL_1294.jpg
Processed NORMAL_538.jpg
Processed NORMAL_1069.jpg
Processed NORMAL_510.jpg
Processed NORMAL_1041.jpg
Processed NORMAL_27

In [3]:
import os
import csv
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel
from skimage.transform import resize
from scipy.fftpack import dct as scipy_dct
import cv2
from skimage.feature import hog

TARGET_SIZE = (128, 128)

kernels = [
    np.real(gabor_kernel(frequency, theta=theta / 4.0 * np.pi, sigma_x=sigma, sigma_y=sigma))
    for theta in range(4)
    for sigma in (1, 3)
    for frequency in (0.05, 0.25)
]

def load_and_preprocess_image(image_path):
    image = img_as_float(io.imread(image_path, as_gray=True)) 
    resized_image = resize(image, TARGET_SIZE, anti_aliasing=True)
    return resized_image

def compute_gabor(image, kernels):
    feats = np.array([(ndi.convolve(image, kernel, mode='wrap').mean(), ndi.convolve(image, kernel, mode='wrap').var())
                      for kernel in kernels])
    return feats.flatten()

def compute_dct(image, num_coeffs=100):
    dct_flat = scipy_dct(scipy_dct(image, axis=0, norm='ortho'), axis=1, norm='ortho').flatten()
    return dct_flat[:num_coeffs]

def compute_ft(image, num_coeffs=100):
    fft_magnitude = np.abs(np.fft.fft2(image)).flatten()
    return np.sort(fft_magnitude)[:num_coeffs]

def compute_phog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), pyramid_levels=3, num_coeffs=100):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    pyramid_features = []
    for level in range(pyramid_levels):
        scaled_image = cv2.resize(image, (0, 0), fx=1 / (2 ** level), fy=1 / (2 ** level))
        hog_features = hog(scaled_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=False, feature_vector=True)
        pyramid_features.extend(hog_features)
    return np.array(pyramid_features)[:num_coeffs]


csv_file_path = 'image_features2.csv'

# Dossier de base contenant les sous-dossiers
base_dir = os.path.expanduser("~/Desktop/LR_grabor/val")
sub_dirs = ["NORMAL", "PNEUMONIA"]

# Ouvrir le fichier CSV en mode écriture
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Numéro d'image", "Gabor", "DCT", "Fourier", "PHOG", "Étiquette"])

    for sub_dir in sub_dirs:
        folder_path = os.path.join(base_dir, sub_dir)
        label = 0 if sub_dir == "NORMAL" else 1

        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(folder_path, filename)
                    
                    # Charger et prétraiter l'image
                    image = load_and_preprocess_image(image_path)
                    
                    # Calculer les caractéristiques
                    gabor_feats = compute_gabor(image, kernels)
                    dct_feats = compute_dct(image, num_coeffs=20)
                    ft_feats = compute_ft(image, num_coeffs=20)
                    phog_feats = compute_phog(image, num_coeffs=20)
                    
                    gabor_mean = np.mean(gabor_feats)
                    dct_mean = np.mean(dct_feats)
                    ft_mean = np.mean(ft_feats)
                    phog_mean = np.mean(phog_feats)

                    
                    # Numéro de l'image
                    numero_image = filename.split('_')[-1].split('.')[0]
                    
                    # Écrire la ligne dans le fichier CSV
                    writer.writerow([numero_image, gabor_mean, dct_mean, ft_mean, phog_mean, label])
                    print(f"Processed {filename}")

print(f"Image features saved to '{csv_file_path}'.")

Processed NORMAL_5218.jpg
Processed NORMAL_5219.jpg
Processed NORMAL_5221.jpg
Processed NORMAL_5220.jpg
Processed NORMAL_5222.jpg
Processed NORMAL_5223.jpg
Processed NORMAL_5217.jpg
Processed NORMAL_5216.jpg
Processed PNEUMONIA_5229.jpg
Processed PNEUMONIA_5228.jpg
Processed PNEUMONIA_5231.jpg
Processed PNEUMONIA_5225.jpg
Processed PNEUMONIA_5224.jpg
Processed PNEUMONIA_5230.jpg
Processed PNEUMONIA_5226.jpg
Processed PNEUMONIA_5227.jpg
Image features saved to 'image_features2.csv'.
